# Maximum Velocity (vmax) vs Accident Probability Analysis

This notebook studies the effect of speed limits (maximum velocity) on accident probability and traffic flow for different injection rates (α).

**Hypothesis**: Higher speed limits may increase throughput but also increase accident severity and probability.

## Import Libraries and Setup

In [ ]:
import sys
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Add parent directory to path to import project modules
sys.path.insert(0, os.path.abspath('..'))

from dispatcher import SimulationDispatcher, generate_parameter_grid

# Set seaborn theme for consistent styling
sns.set_theme(style="whitegrid", palette="Set2")
plt.rcParams['figure.dpi'] = 100

## Simulation Parameters

We fix the following parameters:
- Road length: 100
- Random braking probability (p_b): 0.1
- Lane change probability (p_chg): 0.1
- Red light violation probability (p_red): 0.1
- Skid probability (p_skid): 0
- Green light period (t_green): 40

We vary:
- Injection rate (α): 20 values from 0.05 to 1.0
- Maximum velocity (vmax): {2, 3, 4, 5, 6, 7}

In [ ]:
# Fixed parameters
LENGTH = 100
P_B = 0.1
P_CHG = 0.1
P_RED = 0.1
P_SKID = 0.0
T_GREEN = 40
STEPS = 100000
REPLICATIONS = 5

# Variable parameters
injection_rates = np.linspace(0.05, 1.0, 20)
vmax_values = [2, 3, 4, 5, 6, 7]

print(f"Number of injection rate values: {len(injection_rates)}")
print(f"Injection rates: {injection_rates}")
print(f"vmax values: {vmax_values}")
print(f"Number of replications per configuration: {REPLICATIONS}")
print(f"Total simulations: {len(injection_rates) * len(vmax_values) * REPLICATIONS}")

## Generate Simulation Configurations

In [ ]:
# Generate all parameter combinations
configs = generate_parameter_grid(
    length_values=[LENGTH],
    vmax_values=vmax_values,
    t_green_values=[T_GREEN],
    injection_rate_values=injection_rates.tolist(),
    p_b_values=[P_B],
    p_chg_values=[P_CHG],
    p_red_values=[P_RED],
    p_skid_values=[P_SKID],
    steps=STEPS,
    metrics_start_step=0,
    replications=REPLICATIONS
)

print(f"Generated {len(configs)} simulation configurations")
print(f"\nFirst configuration:")
print(configs[0])

output_file = "vmax_test.csv"

## Run Simulations

This will take some time. The dispatcher will run all simulations in parallel and save results to a CSV file.

In [ ]:
# Create dispatcher and run simulation
dispatcher = SimulationDispatcher(
    output_file=output_file,
    use_multiprocessing=True,
    max_workers=None,  # Use all available CPU cores
    verbose=True
)

# Run all simulations
results = dispatcher.run(configs)

print(f"\nSimulations complete! Results saved to {output_file}")

## Load and Process Results

In [ ]:
# Load results from CSV
df = pd.read_csv(output_file)

# Display basic info
print(f"Total rows: {len(df)}")
print(f"\nColumns: {df.columns.tolist()}")
print(f"\nFirst few rows:")
df.head()

## Calculate Accident Probability

The accident probability (P_acc) is calculated as:

$$P_{acc} = \frac{\text{Number of accidents}}{\text{Time steps} \times \text{Throughput}}$$

Where:
- Number of accidents = n_lateral + n_rear_end (total accidents)
- Time steps = simulation steps
- Throughput = number of cars that passed through the intersection

In [ ]:
# Calculate total accidents and accident probability
df['total_accidents'] = df['n_lateral'] + df['n_rear_end']

# P_acc = Number of accidents / (Time steps * Throughput)
# Avoid division by zero
df['p_acc'] = df.apply(
    lambda row: row['total_accidents'] / (row['steps'] * row['throughput']) 
    if row['throughput'] > 0 else 0, 
    axis=1
)

# Display sample calculations
print("Sample calculations:")
print(df[['injection_rate', 'vmax', 'total_accidents', 'steps', 'throughput', 'p_acc']].head(10))

## Aggregate Results

Average the accident probability across all replications for each combination of injection_rate and vmax.

In [ ]:
# Group by injection_rate and vmax, then calculate mean and std
aggregated = df.groupby(['injection_rate', 'vmax']).agg({
    'p_acc': ['mean', 'std', 'count'],
    'throughput': 'mean',
    'total_accidents': 'mean',
    'avg_speed': 'mean'
}).reset_index()

# Flatten column names
aggregated.columns = ['injection_rate', 'vmax', 'p_acc_mean', 'p_acc_std', 'n_replications', 
                      'throughput_mean', 'accidents_mean', 'avg_speed_mean']

print(f"Aggregated data shape: {aggregated.shape}")
print(f"\nSample aggregated data:")
aggregated.head(10)

## Create Plot: P_acc vs α for Different vmax Values

This plot shows how accident probability varies with injection rate for different maximum velocities (speed limits).

In [ ]:
# Create the plot
plt.figure(figsize=(10, 6))

# Define markers and colors for each vmax value
markers = ['s', 'o', '^', 'v', 'D', 'x']
colors = ['blue', 'green', 'orange', 'red', 'purple', 'brown']

# Plot each vmax value as a separate line
for i, vmax in enumerate(vmax_values):
    data = aggregated[aggregated['vmax'] == vmax]
    
    plt.plot(
        data['injection_rate'], 
        data['p_acc_mean'],
        marker=markers[i],
        color=colors[i],
        linestyle='-',
        linewidth=2,
        markersize=8,
        label=f'vmax={vmax}',
        markerfacecolor=colors[i],
        markeredgecolor=colors[i]
    )

# Formatting
plt.xlabel('α (Injection Rate)', fontsize=14, fontweight='bold')
plt.ylabel('P_acc (Accident Probability)', fontsize=14, fontweight='bold')
plt.title('Accident Probability vs Injection Rate for Different Speed Limits', fontsize=15, fontweight='bold')
plt.xlim(0, 1.0)
plt.ylim(bottom=0)
plt.legend(loc='best', frameon=True, fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()

# Save the figure
plt.savefig('vmax_accident_plot.png', dpi=300, bbox_inches='tight')
print("Plot saved as 'vmax_accident_plot.png'")

plt.show()

## Create Plot: Throughput vs α for Different vmax Values

This plot shows how throughput varies with injection rate for different maximum velocities.

In [ ]:
# Create the plot
plt.figure(figsize=(10, 6))

# Plot each vmax value as a separate line
for i, vmax in enumerate(vmax_values):
    data = aggregated[aggregated['vmax'] == vmax]
    
    plt.plot(
        data['injection_rate'], 
        data['throughput_mean'],
        marker=markers[i],
        color=colors[i],
        linestyle='-',
        linewidth=2,
        markersize=8,
        label=f'vmax={vmax}',
        markerfacecolor=colors[i],
        markeredgecolor=colors[i]
    )

# Formatting
plt.xlabel('α (Injection Rate)', fontsize=14, fontweight='bold')
plt.ylabel('Throughput (vehicles)', fontsize=14, fontweight='bold')
plt.title('Throughput vs Injection Rate for Different Speed Limits', fontsize=15, fontweight='bold')
plt.xlim(0, 1.0)
plt.ylim(bottom=0)
plt.legend(loc='best', frameon=True, fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()

# Save the figure
plt.savefig('vmax_throughput_plot.png', dpi=300, bbox_inches='tight')
print("Plot saved as 'vmax_throughput_plot.png'")

plt.show()

## Create Plot: Average Speed vs α for Different vmax Values

This plot shows how average vehicle speed varies with injection rate for different maximum velocities.

In [ ]:
# Create the plot
plt.figure(figsize=(10, 6))

# Plot each vmax value as a separate line
for i, vmax in enumerate(vmax_values):
    data = aggregated[aggregated['vmax'] == vmax]
    
    plt.plot(
        data['injection_rate'], 
        data['avg_speed_mean'],
        marker=markers[i],
        color=colors[i],
        linestyle='-',
        linewidth=2,
        markersize=8,
        label=f'vmax={vmax}',
        markerfacecolor=colors[i],
        markeredgecolor=colors[i]
    )

# Formatting
plt.xlabel('α (Injection Rate)', fontsize=14, fontweight='bold')
plt.ylabel('Average Speed', fontsize=14, fontweight='bold')
plt.title('Average Speed vs Injection Rate for Different Speed Limits', fontsize=15, fontweight='bold')
plt.xlim(0, 1.0)
plt.ylim(bottom=0)
plt.legend(loc='best', frameon=True, fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()

# Save the figure
plt.savefig('vmax_speed_plot.png', dpi=300, bbox_inches='tight')
print("Plot saved as 'vmax_speed_plot.png'")

plt.show()

## Summary Statistics

Display some key statistics about the results.

In [ ]:
# Find peak accident probability for each vmax
print("Peak accident probability (P_acc) for each vmax value:\n")
for vmax in vmax_values:
    data = aggregated[aggregated['vmax'] == vmax]
    max_idx = data['p_acc_mean'].idxmax()
    max_row = data.loc[max_idx]
    
    print(f"vmax = {vmax}:")
    print(f"  Maximum P_acc = {max_row['p_acc_mean']:.6f} at α = {max_row['injection_rate']:.3f}")
    print(f"  Throughput = {max_row['throughput_mean']:.1f}")
    print(f"  Accidents = {max_row['accidents_mean']:.2f}")
    print(f"  Avg Speed = {max_row['avg_speed_mean']:.2f}")
    print()

# Overall statistics
print("\nOverall statistics:")
print(f"Total simulations run: {len(df)}")
print(f"Total unique configurations: {len(aggregated)}")
print(f"Replications per configuration: {REPLICATIONS}")
print(f"Injection rate range: [{injection_rates.min():.2f}, {injection_rates.max():.2f}]")
print(f"vmax values tested: {vmax_values}")

## Comparison: Safety vs Efficiency Trade-off

Analyze the trade-off between throughput (efficiency) and accident probability (safety) for different speed limits.

In [ ]:
# Create scatter plot: Throughput vs P_acc for each vmax
plt.figure(figsize=(10, 6))

for i, vmax in enumerate(vmax_values):
    data = aggregated[aggregated['vmax'] == vmax]
    
    plt.scatter(
        data['throughput_mean'], 
        data['p_acc_mean'],
        marker=markers[i],
        color=colors[i],
        s=100,
        label=f'vmax={vmax}',
        alpha=0.6,
        edgecolors='black',
        linewidth=0.5
    )

# Formatting
plt.xlabel('Throughput (vehicles)', fontsize=14, fontweight='bold')
plt.ylabel('P_acc (Accident Probability)', fontsize=14, fontweight='bold')
plt.title('Safety-Efficiency Trade-off: Speed Limit Analysis', fontsize=15, fontweight='bold')
plt.legend(loc='best', frameon=True, fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()

# Save the figure
plt.savefig('vmax_tradeoff_plot.png', dpi=300, bbox_inches='tight')
print("Plot saved as 'vmax_tradeoff_plot.png'")

plt.show()

## Interpretation

The analysis reveals several key insights about the relationship between speed limits (vmax) and traffic safety:

### Expected Findings:

1. **Speed Limit Effect on Accidents**: Higher speed limits (larger vmax) generally lead to higher accident probabilities, especially at moderate to high injection rates where traffic density is significant but vehicles can still maintain high speeds.

2. **Throughput vs Safety Trade-off**: Higher speed limits may improve throughput at low-to-moderate injection rates, but this comes at the cost of increased accident risk. The optimal speed limit balances these competing objectives.

3. **Saturation Behavior**: At very high injection rates, the difference between speed limits diminishes as congestion forces all vehicles to slow down regardless of vmax.

4. **Critical Speed Threshold**: There may be a critical speed limit above which accident probability increases dramatically without proportional gains in throughput.

5. **Average Speed Patterns**: Average realized speeds will be lower than vmax, especially at high injection rates, showing that posted speed limits don't directly translate to actual speeds in congested conditions.

### Questions to Explore:

- Is there an optimal vmax that maximizes throughput while keeping accident probability below a safety threshold?
- How does the relationship between vmax and accidents change across different traffic density regimes?
- Do lower speed limits provide a "safety margin" that becomes valuable at high densities?
- What is the marginal safety cost of each unit increase in speed limit?